In [ ]:
!pip install boto3

In [2]:
import json, boto3
from botocore.config import Config

REGION = "af-south-1"
BUCKET = "cct-ds-code-challenge-input-data"
KEY    = "city-hex-polygons-8-10.geojson"

s3 = boto3.client("s3", region_name=REGION, config=Config(s3={"addressing_style": "virtual"}))

sql = """
SELECT s.properties, s.geometry
FROM S3Object[*].features[*] s
WHERE s.properties.resolution = 8
"""

resp = s3.select_object_content(
    Bucket=BUCKET,
    Key=KEY,
    ExpressionType="SQL",
    Expression=sql,
    InputSerialization={"JSON": {"Type": "DOCUMENT"}},
    OutputSerialization={"JSON": {"RecordDelimiter": "\n"}},
)

records = []
for event in resp["Payload"]:
    if "Records" in event:
        payload = event["Records"]["Payload"].decode("utf-8")
        for line in payload.splitlines():
            if line.strip():
                records.append(json.loads(line))
    elif "End" in event:
        break

print("Raw rows:", len(records))
print("First row example:\n", json.dumps(records[0], indent=2)[:500])


ClientError: An error occurred (AccessDenied) when calling the SelectObjectContent operation: User: arn:aws:iam::566800947500:user/code-challenger is not authorized to perform: s3:GetObject on resource: "arn:aws:s3:::cct-ds-code-challenge-input-data/city-hex-polygons-8-10.geojson" with an explicit deny in an identity-based policy